In [2]:
import gradio as gr
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import speech_recognition as sr
from gtts import gTTS
import os
import tempfile

TypeError: type 'typing.TypeVar' is not an acceptable base type

In [ ]:
# Données JSON (50 questions/réponses)
qa_data = [
    {"question": "Quels sont les horaires d'ouverture de la bibliothèque de l'ENIAD Berkane ?", "réponse": "La bibliothèque est ouverte de 8h à 20h du lundi au vendredi, et de 9h à 17h le samedi."},
    {"question": "Comment s'inscrire à un cours d'intelligence artificielle ?", "réponse": "L'inscription se fait en ligne via le portail étudiant ou au bureau des admissions de l'ENIAD."},
    {"question": "Où se trouve le campus de l'ENIAD Berkane ?", "réponse": "Le campus est situé à Berkane, près de l'Université Mohammed 1er, dans la région de l'Oriental."},
    {"question": "Quelles sont les filières proposées à l'ENIAD ?", "réponse": "L'ENIAD propose des filières en IA, Robotique, Sécurité Informatique et Génie Informatique."},
    {"question": "Quel est le coût des frais de scolarité ?", "réponse": "Les frais varient selon le programme ; contactez l'administration pour plus de détails."},
    {"question": "Y a-t-il un internat à l'ENIAD Berkane ?", "réponse": "Non, l'ENIAD ne dispose pas d'internat, mais des logements privés sont disponibles à proximité."},
    {"question": "Comment contacter un professeur ?", "réponse": "Les coordonnées des professeurs sont disponibles sur le portail étudiant ou au secrétariat."},
    {"question": "Quand commence l'année scolaire 2025-2026 ?", "réponse": "L'année scolaire débute généralement en septembre, vérifiez les dates exactes sur le site officiel."},
    {"question": "Quels sont les prérequis pour le cycle ingénieur ?", "réponse": "Un diplôme de niveau bac+2 ou équivalent dans une filière technique est requis."},
    {"question": "L'ENIAD offre-t-elle des formations en ligne ?", "réponse": "Oui, certaines formations courtes sont disponibles en ligne via la plateforme e-learning."},
    {"question": "Comment accéder au Wi-Fi du campus ?", "réponse": "Utilisez votre identifiant étudiant et mot de passe fournis à l'inscription."},
    {"question": "Quels logiciels sont utilisés en cours d'IA ?", "réponse": "Python, TensorFlow, PyTorch et d'autres outils sont couramment utilisés."},
    {"question": "Y a-t-il des stages obligatoires ?", "réponse": "Oui, un stage est requis en fin de cycle pour valider le diplôme d'ingénieur."},
    {"question": "Où puis-je trouver le calendrier des examens ?", "réponse": "Le calendrier est publié sur le portail étudiant au début de chaque semestre."},
    {"question": "L'ENIAD organise-t-elle des événements ?", "réponse": "Oui, des hackathons, conférences et ateliers sont régulièrement organisés."},
    {"question": "Comment demander une bourse ?", "réponse": "Soumettez une demande via le portail étudiant avant la date limite annuelle."},
    {"question": "Quels sont les débouchés après un diplôme de l'ENIAD ?", "réponse": "Les diplômés travaillent dans l'IA, la cybersécurité, le développement logiciel, etc."},
    {"question": "Y a-t-il un club de robotique ?", "réponse": "Oui, le club de robotique est ouvert à tous les étudiants intéressés."},
    {"question": "Comment réserver une salle de travail ?", "réponse": "Les réservations se font via l'application interne de l'ENIAD."},
    {"question": "Quels sont les horaires des cours ?", "réponse": "Les cours ont lieu généralement de 8h30 à 17h, selon l'emploi du temps."},
    {"question": "L'ENIAD a-t-elle un partenariat avec des entreprises ?", "réponse": "Oui, elle collabore avec des entreprises technologiques pour des projets et stages."},
    {"question": "Comment assister à une journée portes ouvertes ?", "réponse": "Les dates sont annoncées sur le site officiel ; l'inscription est souvent requise."},
    {"question": "Quels sont les documents pour l'inscription ?", "réponse": "Carte d'identité, diplômes, relevés de notes et photos d'identité sont nécessaires."},
    {"question": "Y a-t-il des cours en anglais ?", "réponse": "Oui, certains modules avancés sont dispensés en anglais."},
    {"question": "Comment signaler un problème technique ?", "réponse": "Contactez le support informatique via l'adresse email support@eniad.ump.ma."},
    {"question": "L'ENIAD propose-t-elle des certifications ?", "réponse": "Oui, des certifications en IA et cybersécurité sont disponibles."},
    {"question": "Où manger sur le campus ?", "réponse": "Une cafétéria est disponible, mais il n'y a pas de restaurant universitaire."},
    {"question": "Comment participer à un projet de recherche ?", "réponse": "Contactez les professeurs ou le département de recherche de l'ENIAD."},
    {"question": "Quels sont les délais pour payer les frais ?", "réponse": "Les frais doivent être payés avant le début de chaque semestre."},
    {"question": "L'ENIAD a-t-elle une bibliothèque numérique ?", "réponse": "Oui, une bibliothèque numérique est accessible avec vos identifiants étudiants."},
    {"question": "Qu'est-ce que le Big Data à l'ENIAD Berkane ?", "réponse": "Le Big Data à l'ENIAD couvre l'analyse de grandes quantités de données avec des outils comme Hadoop et Spark."},
    {"question": "Quels outils de Big Data sont enseignés ?", "réponse": "Hadoop, Apache Spark, Kafka et des bases de données NoSQL comme MongoDB sont au programme."},
    {"question": "Comment s'inscrire au module Big Data ?", "réponse": "Inscrivez-vous via le portail étudiant ou consultez le responsable du département IA."},
    {"question": "Y a-t-il des projets Big Data à l'ENIAD ?", "réponse": "Oui, les étudiants travaillent sur des projets réels avec des entreprises partenaires."},
    {"question": "Quelles sont les bases de données utilisées en Big Data ?", "réponse": "Les bases vectorielles, MongoDB, et Cassandra sont souvent utilisées."},
    {"question": "Comment accéder aux serveurs Big Data ?", "réponse": "Les serveurs sont accessibles via une connexion sécurisée avec vos identifiants étudiants."},
    {"question": "Le Big Data est-il obligatoire à l'ENIAD ?", "réponse": "Non, mais il est fortement recommandé pour les spécialisations en IA et informatique."},
    {"question": "Quels sont les prérequis pour le cours de Big Data ?", "réponse": "Des bases en programmation (Python) et en bases de données sont nécessaires."},
    {"question": "Y a-t-il des certifications en Big Data ?", "réponse": "Oui, l'ENIAD propose des certifications reconnues en partenariat avec des entreprises."},
    {"question": "Comment analyser des données massives à l'ENIAD ?", "réponse": "Utilisez les outils enseignés comme Spark ou participez aux ateliers pratiques."},
    {"question": "Quels sont les débouchés du Big Data ?", "réponse": "Data Scientist, Ingénieur Big Data, Analyste de données, etc."},
    {"question": "L'ENIAD a-t-elle un cluster Hadoop ?", "réponse": "Oui, un cluster Hadoop est disponible pour les projets étudiants."},
    {"question": "Comment participer à un hackathon Big Data ?", "réponse": "Inscrivez-vous aux événements annoncés sur le portail ou par email."},
    {"question": "Qu'est-ce qu'une base de données vectorielle ?", "réponse": "C'est une base optimisée pour les recherches rapides, utilisée dans le Big Data et l'IA."},
    {"question": "Le Big Data est-il enseigné en anglais ?", "réponse": "Oui, certains cours avancés sont en anglais pour attirer un public international."},
    {"question": "Comment stocker des données massives à l'ENIAD ?", "réponse": "Les données sont stockées sur des serveurs cloud ou locaux avec Hadoop HDFS."},
    {"question": "Y a-t-il des partenariats Big Data avec des entreprises ?", "réponse": "Oui, des collaborations existent avec des sociétés technologiques marocaines et internationales."},
    {"question": "Quels sont les projets Big Data en cours ?", "réponse": "Des projets sur l'analyse de données éducatives et industrielles sont en cours."},
    {"question": "Comment apprendre Apache Spark à l'ENIAD ?", "réponse": "Inscrivez-vous au module Big Data ou suivez les tutoriels en ligne de l'ENIAD."},
    {"question": "Le Big Data aide-t-il les étudiants à l'ENIAD ?", "réponse": "Oui, il permet d'analyser des données pour améliorer les services et l'enseignement."}
]

In [ ]:
# Préparation des données
questions = [item["question"] for item in qa_data]
responses = [item["réponse"] for item in qa_data]
labels = list(range(len(questions)))

In [ ]:
# Vectorisation TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(questions)

In [ ]:
# Entraînement du modèle SVM
model = LinearSVC()
model.fit(X, labels)

LinearSVC()

In [ ]:
# Message d'encouragement
encouragement = "\n\nN’hésitez pas à explorer nos programmes comme l’IA, la robotique ou le Big Data, et à participer à nos hackathons ou ateliers !"

In [ ]:
# Fonction Speech-to-Text
def speech_to_text(audio):
    if audio is None:
        return "Veuillez enregistrer une question avec le microphone."
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio) as source:
        audio_data = recognizer.record(source)
    try:
        text = recognizer.recognize_google(audio_data, language="fr-FR")
        return text
    except sr.UnknownValueError:
        return "Désolé, je n’ai pas compris ce que vous avez dit."
    except sr.RequestError:
        return "Erreur de connexion au service de reconnaissance vocale."

In [ ]:
# Fonction Text-to-Speech
def text_to_speech(text):
    tts = gTTS(text=text, lang="fr", slow=False)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
    tts.save(temp_file.name)
    return temp_file.name

In [ ]:
# Fonction principale du chatbot
def chatbot_response(text_input=None, audio_input=None):
    # Priorité à l'audio si fourni, sinon utiliser le texte
    if audio_input is not None:
        user_input = speech_to_text(audio_input)
    else:
        user_input = text_input if text_input else ""
    
    if not user_input or user_input.strip() == "":
        return "Veuillez poser une question par texte ou voix.", None

    # Vectorisation de la question
    user_vector = vectorizer.transform([user_input])
    similarities = cosine_similarity(user_vector, X)[0]
    best_match_idx = np.argmax(similarities)
    confidence = similarities[best_match_idx]

    # Réponse basée sur la confiance
    if confidence > 0.3:
        response = responses[best_match_idx]
    else:
        response = "Désolé, je ne suis pas sûr de la réponse. Essayez une autre question sur les cours, les clubs ou le Big Data à l’ENIAD !"

    # Ajouter encouragement
    full_response = response + encouragement

    # Générer fichier audio
    audio_file = text_to_speech(full_response)

    return full_response, audio_file


In [ ]:
# Interface Gradio avec audio corrigée
interface = gr.Interface(
    fn=chatbot_response,
    inputs=[
        gr.Textbox(label="Posez votre question ici", placeholder="Tapez ou utilisez le microphone..."),
        gr.Audio(label="Parlez ici", type="filepath")  # Suppression de 'source'
    ],
    outputs=[
        gr.Textbox(label="Réponse du Chatbot"),
        gr.Audio(label="Écoutez la réponse", type="filepath")
    ],
    title="ENIAD Assistant Chatbot avec Parole",
    description="Posez-moi vos questions sur l’ENIAD Berkane par texte ou voix ! Je réponds avec du texte et de l’audio.",
    theme="soft",
    examples=[
        ["Quels sont les horaires d'ouverture de la bibliothèque ?", None],
        ["Y a-t-il des clubs étudiants ?", None],
        ["Qu’est-ce que le Big Data à l’ENIAD ?", None]
    ],
    allow_flagging="never"
)

c:\Users\Abdel\anaconda3\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


In [ ]:
# Lancer l'interface
interface.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Abdel\anaconda3\Lib\site-packages\urllib3\connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Abdel\anaconda3\Lib\site-packages\urllib3\util\connection.py", line 60, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Abdel\anaconda3\Lib\socket.py", line 978, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
socket.gaierror: [Errno 11001] getaddrinfo failed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Users\Abdel\anaconda3\Lib\site-packages\urllib3\connectionpool.py", line 787, in urlopen
    response = self._make_request(
          